## Run and predict model on all images

In [1]:
import torch
from efficientnet_pytorch import EfficientNet
import glob
from PIL import Image
import numpy as np
from fungi_classification import get_transforms
from  tqdm import tqdm


data_dir = "/data/AIDatasets/fungi/DF20M/"
best_trained_model = "/home/cin/Projects/FungiClassification/saved_models/DF20M-EfficientNet-B0_best_accuracy.pth"

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

n_classes = 183
model = EfficientNet.from_name('efficientnet-b0', num_classes=n_classes)
checkpoint = torch.load(best_trained_model)
model.load_state_dict(checkpoint)
model.to(device).eval()


transform = get_transforms("valid")
im_paths = sorted(glob.glob(data_dir + "*.JPG"))
pred_labels = []

for im_path in tqdm(im_paths):
    im = Image.open(im_path)
    im = transform(image=np.asarray(im))
    im = im["image"].to(device).unsqueeze(0)

    with torch.no_grad():
        pred_labels.append(model(im).argmax(1).item())

/home/cin/miniconda3/envs/fungidev/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
100%|██████████| 36393/36393 [25:40<00:00, 23.62it/s]


In [2]:
with open("data_sort_patmjen.npy", "wb") as fp:
    np.savez(fp, im_names=im_paths, pred_labels=pred_labels)

In [1]:
import torch
from efficientnet_pytorch import EfficientNet
import glob
from PIL import Image
import numpy as np
from fungi_classification import get_transforms
from  tqdm import tqdm


data_dir = "/data/AIDatasets/fungi/DF20M/"
best_trained_model = "/home/cin/Projects/FungiClassification/saved_models/DF20M-EfficientNet-B0_best_loss.pth"

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

n_classes = 183
model = EfficientNet.from_name('efficientnet-b0', num_classes=n_classes)
checkpoint = torch.load(best_trained_model)
model.load_state_dict(checkpoint)
model.to(device).eval()


transform = get_transforms("valid")
im_paths = sorted(glob.glob(data_dir + "*.JPG"))
pred_labels = []

batch_sz = 32
c = 0
b_ims = []

with torch.no_grad():
    for im_path in tqdm(im_paths):
        im = Image.open(im_path)
        im = transform(image=np.asarray(im))
        b_ims.append(im["image"])
        c+=1
        if c == batch_sz:
            im = torch.stack(b_ims).to(device)
            c = 0

            with torch.no_grad():
                pred_labels.append(model(im).argmax(1).cpu())
        
    

/home/cin/miniconda3/envs/fungidev/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
  1%|          | 223/36393 [00:07<19:17, 31.25it/s] 


RuntimeError: CUDA out of memory. Tried to allocate 1.78 GiB (GPU 0; 7.80 GiB total capacity; 4.11 GiB already allocated; 1.72 GiB free; 5.13 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [8]:
with torch.no_grad():
    o = model(im)
o.argmax(1).cpu()

tensor([  2, 115, 115, 138,  78,  56, 138, 130,  15, 105,   1, 119,   2,   2,
        107, 116, 107,  90,  83, 163,   0, 107,  10,   0,   0, 163,   0,  53,
         56, 107,  86,   2,   2, 102, 107,   2, 163,   0,   0,   0,   2,   2,
          2,  96,   2,  86,   2, 107,   2,   2, 110,  79, 114,   2,  82,   2,
          2, 171,   1, 107,   0,   2,   2,   2])

## Move images to new dir

In [ ]:
import os
import shutil

out_dir = "/home/cin/Projects/FungiClassification/saved_models/sort_ims"
if not os.path.isdir(out_dir):
    os.mkdir(out_dir)
    for i in range(183):
        os.mkdir(os.path.join(out_dir, str(i)))

for im_path, label in zip(im_paths, pred_labels):
    shutil.copyfile(im_path, os.path.join(out_dir, label, os.path.basename(im_path)))


In [12]:
os.path.basename(out_dir+".jpg")

'sort_ims.jpg'